In [1]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools 
import time
from aima.games import alpha_beta_search, Game, GameState
import copy

In [2]:
def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.terminal_test(state):
        player = state.to_move
        start = time.time()
        move = strategies[player](game, state)
        end = time.time()
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move, 'time: ', end-start, 's.')
            print(state)
    return state

In [3]:
class Tablut(Game):
    def __init__(self, height=9, width=9):
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='WHITE', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('BLACK' if player == 'WHITE' else 'WHITE'))
        win = k_in_row(board, player, square, 9)
        board.utility = (0 if not win else +1 if player == 'WHITE' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'WHITE' else -board.utility

    def terminal_test(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board): print(board)

def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
            for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

In [4]:
class Board(defaultdict):
    empty = '.'
    off = '#'
    
    def __init__(self, width, height, to_move, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)

    def to_move(self, state):
        return self.__dict__['to_move']
        
    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

In [5]:
def random_player(game, state): 
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

In [6]:
infinity = float('inf')

def cache1(function):
    "Like lru_cache(None), but only considers the first argument of function."
    cache = {}
    def wrapped(x, *args):
        if x not in cache:
            cache[x] = function(x, *args)
        return cache[x]
    return wrapped

def cutoff_depth(d):
    """A cutoff function that searches to depth d."""
    return lambda game, state, depth: depth > d

def h_alphabeta_search(game, state, cutoff=cutoff_depth(5), h=lambda s, p: 0):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move

    @cache1
    def max_value(state, alpha, beta, depth):
        if game.terminal_test(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta, depth+1)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    @cache1
    def min_value(state, alpha, beta, depth):
        if game.terminal_test(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta, depth + 1)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity, 0)

In [7]:
%%time
play_game(Tablut(), dict(WHITE=player(h_alphabeta_search), BLACK=player(h_alphabeta_search)), verbose=True).utility

Player WHITE move: (4, 0) time:  19.686488151550293 s.
. . . . WHITE . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .

Player BLACK move: (3, 7) time:  20.44010090827942 s.
. . . . WHITE . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . BLACK . . . . .
. . . . . . . . .

Player WHITE move: (5, 4) time:  19.947776079177856 s.
. . . . WHITE . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . WHITE . . .
. . . . . . . . .
. . . . . . . . .
. . . BLACK . . . . .
. . . . . . . . .

Player BLACK move: (4, 6) time:  20.718076705932617 s.
. . . . WHITE . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . WHITE . . .
. . . . . . . . .
. . . . BLACK . . . .
. . . BLACK . . . . .
. . . . . . . . .

Player WHITE move: (5, 1) time:  19.96269178390503 s.
. . . . WHITE . . . .
. . . . . WHI

KeyboardInterrupt: 